In [1]:
import scanpy as sc
import numpy as np
np.random.seed(15)
import os
import scipy as sp

In [3]:
from scipy.stats import bernoulli
def naive_dropout(X, norm_loc=0.1, norm_scale=0.5, plt_orig=False):
    N, G = X.shape
    norm_loc_log = np.log(norm_loc)
    # loc = mean, scale = standard deviation, size=number of integer
    dropout_prob_per_gene = np.exp(np.random.normal(loc=norm_loc_log, scale=norm_scale, size=G))
    dropped_out = np.zeros((N, G), dtype=np.bool)
    dropout_picker = bernoulli(dropout_prob_per_gene)
    for n in range(N):
        dropped_out[n] = dropout_picker.rvs()

    drop_X = np.copy(X)
    drop_X[dropped_out] = 0
    return drop_X

# Simulate additional levels of sparsity

## 70% Sparsity

In [4]:
data_dir = 'D:/Dimensionality Reduction Project additional files/Simulated/Raw/'
Cont_abund = sc.read_h5ad(f'{data_dir}Continuous_Abundant_Simulation_Raw.h5ad')
Disc_abund = sc.read_h5ad(f'{data_dir}Discrete_Abundant_Simulation_Raw.h5ad')
Cont_rare = sc.read_h5ad(f'{data_dir}Continuous_OnlyRare_Simulation_Raw.h5ad')
Disc_rare = sc.read_h5ad(f'{data_dir}Discrete_OnlyRare_Simulation_Raw.h5ad')
Cont_ultrarare = sc.read_h5ad(f'{data_dir}Continuous_UltraRare_Simulation_Raw.h5ad')
Disc_ultrarare = sc.read_h5ad(f'{data_dir}Discrete_UltraRare_Simulation_Raw.h5ad')

In [5]:
datasets = {
    'Discrete Abundant': Disc_abund,
    'Discrete Moderately-Rare': Disc_rare,
    'Discrete Ultra-Rare': Disc_ultrarare,
    'Continuous Abundant': Cont_abund,
    'Continuous Moderately-Rare': Cont_rare,
    'Continuous Ultra-Rare': Cont_ultrarare
}

### Check Sparsity of raw datasets

In [6]:
for dataset in datasets.keys():
    if sp.sparse.isspmatrix_csr(datasets[dataset].X) == False: #Convert all to CSR if not already
        datasets[dataset].X = sp.sparse.csr_matrix(datasets[dataset].X)

In [7]:
for dataset in datasets.keys():
    n_cells_detected = datasets[dataset].X.getnnz(axis=0)
    pct_non_zero = n_cells_detected.sum()/(datasets[dataset].X.shape[0]*datasets[dataset].X.shape[1])*100
    print("percent of zero values is ", round(100 - pct_non_zero,2), "% for", dataset)
    print("Number of genes universally expressed in all cells is ",sum(n_cells_detected == datasets[dataset].X.shape[0]))

percent of zero values is  49.79 % for Discrete Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  49.39 % for Discrete Moderately-Rare
Number of genes universally expressed in all cells is  1
percent of zero values is  48.85 % for Discrete Ultra-Rare
Number of genes universally expressed in all cells is  1
percent of zero values is  48.31 % for Continuous Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  46.5 % for Continuous Moderately-Rare
Number of genes universally expressed in all cells is  1
percent of zero values is  46.08 % for Continuous Ultra-Rare
Number of genes universally expressed in all cells is  1


In [8]:
for dataset in datasets.keys():
    norm_locs = [0.42]
    norm_scales = [0.1]
    for i, norm_scale in enumerate(norm_scales):
        drop_X = naive_dropout(datasets[dataset].X.todense(), norm_loc=norm_locs[i], norm_scale=norm_scale, plt_orig=False)
        drop_X = sp.sparse.csr_matrix(drop_X)
        n_cells_detected = drop_X.getnnz(axis=0)
        pct_non_zero = n_cells_detected.sum() / (drop_X.shape[0] * drop_X.shape[1]) * 100
        print(norm_scale, norm_locs[i])
        print("percent of zero values is ", round(100 - pct_non_zero, 2), "% for", dataset)
        print("Number of genes universally expressed in all cells is ",
            sum(n_cells_detected == drop_X.shape[0]))
    datasets[dataset].X = drop_X
    datasets[dataset].write(os.path.join(data_dir, dataset + '_5kG_10C_Simulation_70%_Raw.h5ad'))

C:\Users\uqewats6\AppData\Local\Temp/ipykernel_15436/2674768251.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dropped_out = np.zeros((N, G), dtype=np.bool)


0.1 0.42
percent of zero values is  71.01 % for Discrete Abundant
Number of genes universally expressed in all cells is  0
0.1 0.42
percent of zero values is  70.75 % for Discrete Moderately-Rare
Number of genes universally expressed in all cells is  0
0.1 0.42
percent of zero values is  70.4 % for Discrete Ultra-Rare
Number of genes universally expressed in all cells is  0
0.1 0.42
percent of zero values is  70.14 % for Continuous Abundant
Number of genes universally expressed in all cells is  0
0.1 0.42
percent of zero values is  69.13 % for Continuous Moderately-Rare
Number of genes universally expressed in all cells is  0
0.1 0.42
percent of zero values is  68.85 % for Continuous Ultra-Rare
Number of genes universally expressed in all cells is  0


### Data Processing for 70%

In [11]:
for dataset in datasets.keys():
    if sp.sparse.isspmatrix_csr(datasets[dataset].X) == False: #Convert all to CSR if not already
        datasets[dataset].X = sp.sparse.csr_matrix(datasets[dataset].X)
    sc.pp.calculate_qc_metrics(datasets[dataset], percent_top=None, log1p=False, inplace=True)
    sc.pp.filter_cells(datasets[dataset], min_genes=200, inplace=True) #Filter cells expression >200 genes
    sc.pp.filter_genes(datasets[dataset], min_cells=(len(datasets[dataset].obs)*0.1), inplace=True) #Filter genes expressed in <10% of cells
    sc.pp.normalize_total(datasets[dataset], target_sum=1e4, key_added='norm_factor')
    sc.pp.log1p(datasets[dataset])

### Check sparsity after processing

In [12]:
for dataset in datasets.keys():
    n_cells_detected = datasets[dataset].X.getnnz(axis=0)
    pct_non_zero = n_cells_detected.sum()/(datasets[dataset].X.shape[0]*datasets[dataset].X.shape[1])*100
    print("percent of zero values is ", round(100 - pct_non_zero,2), "% for", dataset)
    print("Number of genes universally expressed in all cells is ",sum(n_cells_detected == datasets[dataset].X.shape[0]))

percent of zero values is  67.73 % for Discrete Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  67.0 % for Discrete Moderately-Rare
Number of genes universally expressed in all cells is  0
percent of zero values is  66.51 % for Discrete Ultra-Rare
Number of genes universally expressed in all cells is  0
percent of zero values is  66.69 % for Continuous Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  65.92 % for Continuous Moderately-Rare
Number of genes universally expressed in all cells is  0
percent of zero values is  65.74 % for Continuous Ultra-Rare
Number of genes universally expressed in all cells is  0


### Save processed dataset

In [ ]:
for dataset in datasets.keys():
    datasets[dataset].write(os.path.join(data_dir, dataset + '_Simulation_70%_Processed.h5ad'))

## 90% Sparsity

In [40]:
data_dir = 'D:/Dimensionality Reduction Project additional files/Simulated/Raw/'
Cont_abund = sc.read_h5ad(f'{data_dir}Continuous_Abundant_Simulation_Raw.h5ad')
Disc_abund = sc.read_h5ad(f'{data_dir}Discrete_Abundant_Simulation_Raw.h5ad')
Cont_rare = sc.read_h5ad(f'{data_dir}Continuous_OnlyRare_Simulation_Raw.h5ad')
Disc_rare = sc.read_h5ad(f'{data_dir}Discrete_OnlyRare_Simulation_Raw.h5ad')
Cont_ultrarare = sc.read_h5ad(f'{data_dir}Continuous_UltraRare_Simulation_Raw.h5ad')
Disc_ultrarare = sc.read_h5ad(f'{data_dir}Discrete_UltraRare_Simulation_Raw.h5ad')

In [41]:
datasets = {
    'Discrete Abundant': Disc_abund,
    'Discrete Moderately-Rare': Disc_rare,
    'Discrete Ultra-Rare': Disc_ultrarare,
    'Continuous Abundant': Cont_abund,
    'Continuous Moderately-Rare': Cont_rare,
    'Continuous Ultra-Rare': Cont_ultrarare
}

### Check Sparsity of raw datasets

In [42]:
for dataset in datasets.keys():
    if sparse.isspmatrix_csr(datasets[dataset].X) == False: #Convert all to CSR if not already
        datasets[dataset].X = sparse.csr_matrix(datasets[dataset].X)

In [43]:
for dataset in datasets.keys():
    n_cells_detected = datasets[dataset].X.getnnz(axis=0)
    pct_non_zero = n_cells_detected.sum()/(datasets[dataset].X.shape[0]*datasets[dataset].X.shape[1])*100
    print("percent of zero values is ", round(100 - pct_non_zero,2), "% for", dataset)
    print("Number of genes universally expressed in all cells is ",sum(n_cells_detected == datasets[dataset].X.shape[0]))

percent of zero values is  49.79 % for Discrete Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  49.39 % for Discrete Moderately-Rare
Number of genes universally expressed in all cells is  1
percent of zero values is  48.85 % for Discrete Ultra-Rare
Number of genes universally expressed in all cells is  1
percent of zero values is  48.31 % for Continuous Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  46.5 % for Continuous Moderately-Rare
Number of genes universally expressed in all cells is  1
percent of zero values is  46.08 % for Continuous Ultra-Rare
Number of genes universally expressed in all cells is  1


In [44]:
for dataset in datasets.keys():
    norm_locs = [0.8]
    norm_scales = [0.01]
    for i, norm_scale in enumerate(norm_scales):
        drop_X = naive_dropout(datasets[dataset].X.todense(), norm_loc=norm_locs[i], norm_scale=norm_scale, plt_orig=False)
        drop_X = sp.sparse.csr_matrix(drop_X)
        n_cells_detected = drop_X.getnnz(axis=0)
        pct_non_zero = n_cells_detected.sum() / (drop_X.shape[0] * drop_X.shape[1]) * 100
        print(norm_scale, norm_locs[i])
        print("percent of zero values is ", round(100 - pct_non_zero, 2), "% for", dataset)
        print("Number of genes universally expressed in all cells is ",
            sum(n_cells_detected == drop_X.shape[0]))
    datasets[dataset].X = drop_X
    datasets[dataset].write(os.path.join(data_dir, dataset + '_5kG_10C_Simulation_90%_Raw.h5ad'))

C:\Users\uqewats6\AppData\Local\Temp/ipykernel_14652/2674768251.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dropped_out = np.zeros((N, G), dtype=np.bool)


0.01 0.8
percent of zero values is  89.97 % for Discrete Abundant
Number of genes universally expressed in all cells is  0
0.01 0.8
percent of zero values is  89.85 % for Discrete Moderately-Rare
Number of genes universally expressed in all cells is  0
0.01 0.8
percent of zero values is  89.77 % for Discrete Ultra-Rare
Number of genes universally expressed in all cells is  0
0.01 0.8
percent of zero values is  89.66 % for Continuous Abundant
Number of genes universally expressed in all cells is  0
0.01 0.8
percent of zero values is  89.31 % for Continuous Moderately-Rare
Number of genes universally expressed in all cells is  0
0.01 0.8
percent of zero values is  89.22 % for Continuous Ultra-Rare
Number of genes universally expressed in all cells is  0


### Data Processing for 90%

In [34]:
for dataset in datasets.keys():
    if sparse.isspmatrix_csr(datasets[dataset].X) == False: #Convert all to CSR if not already
        datasets[dataset].X = sparse.csr_matrix(datasets[dataset].X)
    sc.pp.calculate_qc_metrics(datasets[dataset], percent_top=None, log1p=False, inplace=True)
    sc.pp.filter_cells(datasets[dataset], min_genes=200, inplace=True) #Filter cells expression >200 genes
    sc.pp.filter_genes(datasets[dataset], min_cells=(len(datasets[dataset].obs)*0.1), inplace=True) #Filter genes expressed in <10% of cells
    sc.pp.normalize_total(datasets[dataset], target_sum=1e4, key_added='norm_factor')
    sc.pp.log1p(datasets[dataset])

### Check sparsity after processing

In [36]:
for dataset in datasets.keys():
    n_cells_detected = datasets[dataset].X.getnnz(axis=0)
    pct_non_zero = n_cells_detected.sum()/(datasets[dataset].X.shape[0]*datasets[dataset].X.shape[1])*100
    print("percent of zero values is ", round(100 - pct_non_zero,2), "% for", dataset)
    print("Number of genes universally expressed in all cells is ",sum(n_cells_detected == datasets[dataset].X.shape[0]))

percent of zero values is  67.67 % for Discrete Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  67.02 % for Discrete Moderately-Rare
Number of genes universally expressed in all cells is  0
percent of zero values is  66.54 % for Discrete Ultra-Rare
Number of genes universally expressed in all cells is  0
percent of zero values is  66.6 % for Continuous Abundant
Number of genes universally expressed in all cells is  0
percent of zero values is  65.98 % for Continuous Moderately-Rare
Number of genes universally expressed in all cells is  0
percent of zero values is  65.68 % for Continuous Ultra-Rare
Number of genes universally expressed in all cells is  0


### Save processed dataset

In [ ]:
for dataset in datasets.keys():
    datasets[dataset].write(os.path.join(data_dir, dataset + '_Simulation_90%_Processed.h5ad'))